**COMPREHENSIVE STRENGTH OF CONCRETE**

In [1]:
# IMPORT LIBRARIES
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 
%matplotlib inline 

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor)
from sklearn.svm import SVR
form sklearn import metrics 
from sklearn.ensemble import Voi=tingRegressor
from scipy import stats 
from scipt.stats import zscore 
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import 
